<a href="https://colab.research.google.com/github/Manish-k723/Movie-recommedation-system/blob/main/Book_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
!pip install opendatasets --quiet

In [71]:
import opendatasets as od

In [72]:
import pandas as pd
import numpy as np

In [73]:
url = 'https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata?select=tmdb_5000_movies.csv'

In [74]:
od.download(url)
#user_name = manishkumar7432698
#api = 85fb7bae22551948ed36c1a6d8da72a9

Skipping, found downloaded files in "./tmdb-movie-metadata" (use force=True to force download)


In [75]:
movie_df = pd.read_csv('/content/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits_df = pd.read_csv('/content/tmdb-movie-metadata/tmdb_5000_credits.csv')

In [76]:
movie_df.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
4498,600000,"[{""id"": 35, ""name"": ""Comedy""}]",NaN,11598,"[{""id"": 917, ""name"": ""journalism""}, {""id"": 100...",en,The Kentucky Fried Movie,A series of loosely connected skits that spoof...,10.861167,"[{""name"": ""Kentucky Fried Theatre"", ""id"": 3894}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1977-08-10,0,83.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,This movie is totally out of control!,The Kentucky Fried Movie,6.4,66
1019,55000000,"[{""id"": 37, ""name"": ""Western""}]",http://www.310toyumathefilm.com/,5176,"[{""id"": 1582, ""name"": ""saloon""}, {""id"": 1701, ...",en,3:10 to Yuma,"In Arizona in the late 1800's, infamous outlaw...",48.801089,"[{""name"": ""Tree Line Films"", ""id"": 84}, {""name...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-09-06,70016220,122.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Time waits for one man.,3:10 to Yuma,6.9,1188
2917,25000000,"[{""id"": 18, ""name"": ""Drama""}]",http://www.thefightermovie.com,45317,"[{""id"": 6075, ""name"": ""sport""}, {""id"": 154123,...",en,The Fighter,"The Fighter, is a drama about boxer ""Irish"" Mi...",31.019381,"[{""name"": ""Paramount Pictures"", ""id"": 4}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2010-12-17,93617009,116.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Every dream deserves a fighting chance.,The Fighter,7.2,1486
1153,40000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 878, ""na...",http://lucymovie.com/,240832,"[{""id"": 310, ""name"": ""artificial intelligence""...",en,Lucy,"A woman, accidentally caught in a dark deal, t...",115.241998,"[{""name"": ""Universal Pictures"", ""id"": 33}, {""n...","[{""iso_3166_1"": ""FR"", ""name"": ""France""}, {""iso...",2014-07-14,126546825,89.0,"[{""iso_639_1"": ""zh"", ""name"": ""\u666e\u901a\u8b...",Released,The average person uses 10% of their brain cap...,Lucy,6.3,5878
91,165000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.warof1996.com,47933,"[{""id"": 12026, ""name"": ""alternate history""}, {...",en,Independence Day: Resurgence,We always knew they were coming back. Using re...,48.775723,"[{""name"": ""Twentieth Century Fox Film Corporat...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-06-22,389681935,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,We had twenty years to prepare. So did they.,Independence Day: Resurgence,4.9,2491


The data is cluttered. It has lot of information but not organized well and it make difficult for us to analyze it. If we directly provide this data to the model, model will get confused in giving weights to attributes.Hence, We need preprocessing for extracting important and useful information.

In [77]:
credits_df.sample(3) 

,movie_id,title,cast,crew
1141,65,8 Mile,"[{""cast_id"": 3, ""character"": ""Jimmy B. \""Rabbi...","[{""credit_id"": ""52fe4212c3a36847f8001c6b"", ""de..."
4589,43743,Fabled,[],[]
102,131634,The Hunger Games: Mockingjay - Part 2,"[{""cast_id"": 2, ""character"": ""Katniss Everdeen...","[{""credit_id"": ""551e57929251411d8d00029a"", ""de..."


In [78]:
credits_df['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

Our metadata dataset is also cluttered and diffiult to analyze.
Preprocesssing is required.

The first thing is we should merge the two datasets. 

In [79]:
movie = movie_df.merge(credits_df, on = 'title')

In [80]:
movie.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [81]:
movie.shape

(4809, 23)

Our data has lot of information and attributes and in some of the attributes also we have lot of data. Therefore,we need some relaxation from too much information. We will filter out some data according to our requirement means the data which help us in recommendation system.

Like no-one search a movie by its camera man name or any secondary crew. Hence, we will remove this data from our data set.

So, here is the list which is required for recommending a movie to a user which is based on user searches and personal experience.


1.   Genre
2.   id ( not for recommendation but for setting up a website it is important
3.   keywords
4.   title
5.   overview
6.   production_company( will use it later currently it will create complexity)
7.   release_date(currently not using but an important factor)
8.   cast
9.   crew(due to director)

In [82]:
movies = movie[['movie_id','title','genres','keywords','overview','cast','crew']]

To create a recommendation system, I will create a new dataframe in which we will have only 3 attributes
1.   movie_id
2.   title
3.   Tag

In tag, we will have combination of all other columns apart from movie_id and title from movies dataset. But for this purpose, first I need to clean different columns and extract only required keywords from those attributes.



## **Data Preprocessing**

Let's do the above mentioned but to perform that, we need to clean and reduce cluterness of data.

### **Handling Null values**

In [83]:
movies.isna().sum()

movie_id    0
title       0
genres      0
keywords    0
overview    3
cast        0
crew        0
dtype: int64

In [84]:
movies = movies.dropna()

In [85]:
movies.isna().sum()

movie_id    0
title       0
genres      0
keywords    0
overview    0
cast        0
crew        0
dtype: int64

### **Checking** **duplicates**

In [86]:
movies.duplicated().sum()

0

### **Formatting of attributes**

In [87]:
movies.head(1)

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [88]:
import ast

ast module converts string to list. 

In [89]:
def extract(obj):
  l = []
  for i in ast.literal_eval(obj):
    l.append(i['name'])
  return l

In [90]:
movies['genres'] = movies.genres.apply(extract)

In [91]:
movies['keywords'] = movies.keywords.apply(extract)

In [92]:
movies.cast[0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [94]:
def extract_actor(obj):
  l = []
  count = 0
  for i in ast.literal_eval(obj):
    if count < 3:
      l.append(i['name'])
      count+=1
    else:
      break
  return l

In [95]:
movies.cast = movies.cast.apply(extract_actor)

In [96]:
movies.crew[0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

I have to only extract the name of director.

In [99]:
def extract_director(obj):
  l = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      l.append(i['name'])
      break
  return l

In [101]:
movies['crew'] = movies['crew'].apply(extract_director)

In [106]:
movies.overview = movies.overview.apply(lambda x:x.split())

In [108]:
movies.head()

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[In, the, 22nd, century,, a, paraplegic, Marin...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Captain, Barbossa,, long, believed, to, be, d...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[A, cryptic, message, from, Bond’s, past, send...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Following, the, death, of, District, Attorney...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[John, Carter, is, a, war-weary,, former, mili...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


## **Removing White space**
One more thing, I have to do here is to remove white spaces between the elements of any list in our dataset. 

The reason why I am doing this is that to remove confusion to our model. Let's take the very first actor name which is `sam worthington` and the name of director of movie spectre is is `sam mendes`. If someone puts sam worthington in our recommender system then our system will search firstly for sam and also recommend a movie which is not of our consumers use. So to cure it, we will remove white spaces.

In [118]:
def remove_space(columns):
  for column in columns:
    movies[column] = movies[column].apply(lambda x: [i.replace(" ",'') for i in x])

In [119]:
columns = ['genres','keywords','cast','crew']
remove_space(columns)

In [120]:
movies.head()

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


## **Creation of Tag column**
Last thing to do for processing is to create a tag column which is combination genres, keywords, overview, cast, crew. The tag column will be in small cap and in form of string.

In [135]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [136]:
new_df = movies[['movie_id','title','tags']]

In [137]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

<ipython-input-137-e498c0e361a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))


In [138]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

<ipython-input-138-db70cfe1859e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())


In [140]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


Here, we need to do stemming to avoid repetition of words in different forms in most common words. It will improve the accuracy very much.

In [161]:
import nltk

In [163]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [180]:
def stem(text):
  y = []

  for i in text.split():
    y.append(ps.stem(i))
  return ' '.join(y)

In [184]:
new_df['tags'] = new_df.tags.apply(stem)

<ipython-input-184-33874492b6a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df.tags.apply(stem)


In [185]:
new_df

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just want to play hi guitar and ca...
4805,72766,Newlyweds,a newlyw couple' honeymoon is upend by the arr...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduc a dedic q..."
4807,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...


## **Modelling**
How our model will work - 

1. Create a bag of words using all available text.
2. Extract the most common words to be used as the vectorizer.
3. Pass the tags column to the vectorizer.
4. Convert the defined number of text (5000) in the vectorizer into column headings, with the movie names as rows.
5. Analyze the movies and assign a value of 1 to each column heading that is present in the movie tag text.
6. Plot a graph based on the analysis.
7. If someone searches for a word, suggest the 5 or 10 closest points on the graph to the searched word.

In [186]:
from sklearn.feature_extraction.text import CountVectorizer

In [187]:
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [188]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [189]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

This is our points which we will plot on the graph, every row of this matrix represent the matched words in the first movie and the most common words in our bag of words.

In [190]:
cv.get_feature_names()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000',
 '007',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '17th',
 '18',
 '18th',
 '18thcenturi',
 '19',
 '1910',
 '1920',
 '1930',
 '1940',
 '1944',
 '1950',
 '1950s',
 '1960',
 '1960s',
 '1970',
 '1970s',
 '1971',
 '1974',
 '1976',
 '1980',
 '1985',
 '1990',
 '1999',
 '19th',
 '19thcenturi',
 '20',
 '200',
 '2003',
 '2009',
 '20th',
 '21st',
 '23',
 '24',
 '25',
 '30',
 '300',
 '3d',
 '40',
 '50',
 '500',
 '60',
 '70',
 '80',
 'aaron',
 'aaroneckhart',
 'abandon',
 'abduct',
 'abigailbreslin',
 'abil',
 'abl',
 'aboard',
 'abov',
 'abus',
 'academ',
 'academi',
 'accept',
 'access',
 'accid',
 'accident',
 'acclaim',
 'accompani',
 'accomplish',
 'account',
 'accus',
 'ace',
 'achiev',
 'acquaint',
 'act',
 'action',
 'actionhero',
 'activ',
 'activist',
 'activities',
 'actor',
 'actress',
 'actual',
 'ad',
 'adam',
 'adamsandl',
 'adamshankman',
 'adapt',
 'add',
 'addict',
 'adjust',
 'admir',
 'admit',
 'adolesc',
 'adopt',
 'ador',
 'adrienbrodi',
 'adult'

Improvement on 1st run - Now, We will apply stemming to remove or merge similar words. Like in our data words such as actor and actors are treated differently but they are same. So to remove this we do stemming.So, we have to go back for now to remove these words from the root column which was nltk. 

In [191]:
vectors.shape

(4806, 5000)

In this shape, 4806 is representing the number of movies available to the machine and 5000 is representing the most common 5000 words from the bag of words
.

## **Distance Calculation**
After vectorization is calculating the distance between the movies on this 5000*5000 dimensional graph. It will help us to suggest the similar movies. Here, we will use cosine distane not the eucleadian distance because the more the dimension, the bad the eucleadean distance for the method.

In [192]:
from sklearn.metrics.pairwise import cosine_similarity

In [194]:
similarity = cosine_similarity(vectors)

This array is representing the distance(dissimilarity) of movies with each other. Its shape is 4806X4806 which is verifying the statement.

In [195]:
similarity

array([[1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
        0.        ],
       [0.08346223, 1.        , 0.06063391, ..., 0.02378257, 0.        ,
        0.02615329],
       [0.0860309 , 0.06063391, 1.        , ..., 0.02451452, 0.        ,
        0.        ],
       ...,
       [0.04499213, 0.02378257, 0.02451452, ..., 1.        , 0.03962144,
        0.04229549],
       [0.        , 0.        , 0.        , ..., 0.03962144, 1.        ,
        0.08714204],
       [0.        , 0.02615329, 0.        , ..., 0.04229549, 0.08714204,
        1.        ]])

The similarity of a movie with itself is obviously one and that is represented by the diagonals.

## **Recommendation Function**

In [223]:
def recommend(movie):
  movie_index = new_df[new_df['title'] == movie].index[0]
  similar = sorted(list(enumerate(similarity[movie_index])),reverse = True,key = lambda x: x[1])[1:6]
  for i in similar:
    print(new_df.iloc[i[0]].title)

In [224]:
recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [225]:
import pickle

In [226]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [227]:
pickle.dump(similarity,open('similarity.pkl','wb'))